# Tutorial, Part I: Candidate Extraction

In this example, we'll be writing an application to extract **chemical-induced-disease relationships** from Pubmed abstracts, as per the [BioCreative CDR Challenge](http://www.biocreative.org/resources/corpora/biocreative-v-cdr-corpus/).  At core, we will be constructing a model to classify _candidate chemical-disease (C-D) relation mentions_ as either true or false.  To do this, we first need a set of such candidates- in this notebook, we'll use `DDLite` utilities to extract these candidates.

## Loading the Corpus

First, we will load and pre-process the corpus, storing it for convenience in a `Corpus` object

### Configuring a document parser

We'll start by defining a `DocParser` class to read in Pubmed abstracts from [Pubtator]([Pubtator](http://www.ncbi.nlm.nih.gov/CBBresearch/Lu/Demo/PubTator/index.cgi)), where they are stored along with "gold" (i.e. hand-annotated by experts) *chemical* and *disease mention* annotations. We'll use the `XMLDocParser` class, which allows us to use [XPath queries](https://en.wikipedia.org/wiki/XPath) to specify the relevant sections of the XML format.

_Note that we are newline-concatenating text from the title and abstract together for simplicity, but if we wanted to, we could easily extend the `DocParser` classes to preserve information about document structure._

In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from ddlite_parser import XMLDocParser
xml_parser = XMLDocParser(
    path='data/CDR_DevelopmentSet.xml',
    doc='.//document',
    text='.//passage/text/text()',
    id='.//id/text()',
    keep_xml_tree=True)

### Selecting a sentence parser

Next, we'll use an NLP preprocessing tool to split the `Document` objects into sentences, tokens, and provide annotations--part-of-speech tags, dependency parse structure, lemmatized word forms, etc.--for these sentences.  Here we use the default `SentenceParser` class.

In [3]:
from ddlite_parser import SentenceParser
sent_parser = SentenceParser()

### Pre-processing & loading the corpus

Finally, we'll put this all together using a `Corpus` object, which will execute the parsers and store the results as an iterator:

In [5]:
from ddlite_parser import Corpus
%time corpus = Corpus(xml_parser, sent_parser)

Parsing documents...
Parsing contexts...
CPU times: user 4.51 s, sys: 151 ms, total: 4.66 s
Wall time: 35.5 s


In [6]:
doc  = corpus.get_docs()[2]
print doc

Document(id='1445986', file='CDR_DevelopmentSet.xml', text="Cefotetan-induced immune hemolytic anemia.\nImmune hemolytic anemia due to a drug-adsorption mechanism has been described primarily in patients receiving penicillins and first-generation cephalosporins. We describe a patient who developed anemia while receiving intravenous cefotetan. Cefotetan-dependent antibodies were detected in the patient's serum and in an eluate prepared from his red blood cells. The eluate also reacted weakly with red blood cells in the absence of cefotetan, suggesting the concomitant formation of warm-reactive autoantibodies. These observations, in conjunction with clinical and laboratory evidence of extravascular hemolysis, are consistent with drug-induced hemolytic anemia, possibly involving both drug-adsorption and autoantibody formation mechanisms. This case emphasizes the need for increased awareness of hemolytic reactions to all cephalosporins.", attribs={'root': <Element document at 0x103da89e0>}

In [7]:
sent = corpus.get_sentences_in(doc.id)[0]
print sent

Sentence(id='1445986-0', doc_id='1445986', doc_name='CDR_DevelopmentSet.xml', sent_id=0, words=[u'Cefotetan-induced', u'immune', u'hemolytic', u'anemia', u'.'], lemmas=[u'cefotetan-induced', u'immune', u'hemolytic', u'anemia', u'.'], poses=[u'JJ', u'JJ', u'JJ', u'NN', u'.'], dep_parents=[4, 4, 4, 0, 4], dep_labels=[u'amod', u'amod', u'amod', u'ROOT', u'punct'], char_offsets=[0, 18, 25, 35, 41], text=u'Cefotetan-induced immune hemolytic anemia.')


## Writing a basic candidate extractor

Next, we'll write a basic function to extract **candidate disease mentions** from the corpus.  For this first attempt, we'll just write a function that checks for matches against a list (or _"dictionary"_) of disease phrases, constructed using some pre-compiled ontologies ([UMLS](https://www.nlm.nih.gov/research/umls/), [ORDO](http://www.orphadata.org/cgi-bin/inc/ordo_orphanet.inc.php), [DOID](http://www.obofoundry.org/ontology/doid.html), [NCBI Diseases](http://www.ncbi.nlm.nih.gov/CBBresearch/Dogan/DISEASE/); see `tutorial/data/diseases.py`).

We'll do this using a `CandidateSpace` object--which defines the basic candidates we consider, in this case n-grams up to a certain length--and a `Matcher` object, which filters this candidate space down.

In [23]:
from load_dictionaries import load_disease_dictionary
from ddlite_candidates import Ngrams
from ddlite_matchers import DictionaryMatch

# Load the disease phrase dictionary
diseases = load_disease_dictionary()
print "Loaded %s disease phrases!" % len(diseases)

# Define a candidate space
ngrams = Ngrams(n_max=3)

# Define a matcher
matcher = DictionaryMatch(d=diseases, longest_match_only=False)

Loaded 507899 disease phrases!


Note that we set `longest_match_only=False`, which means that we _will_ consider subsequences of phrases that match our dictionary.

The `Ngrams` operator is applied over our `Sentence` objects and returns `Ngram` objects, and the `Matcher` then filters these, so we apply our operators over the sentences in the corpus, storing the results in a `Candidates` object for convenience:

In [46]:
from ddlite_candidates import Candidates
%time c = Candidates(ngrams, matcher, corpus.get_sentences())
a = c.get_candidates()[0]

Extracting candidates...
CPU times: user 27.6 s, sys: 534 ms, total: 28.1 s
Wall time: 27.8 s


In [47]:
a.get_attrib_tokens("dep_labels")

[u'amod', u'nmod']

## Evaluating our candidate recall on gold annotations

Next, we'll test our _candidate recall_--in other words, how many of the true disease mentions we picked up in our candidate set--using the gold annotations in our dataset.

The XML documents that we loaded using the `XMLDocParser` also contained annotations (this is why we kept the full xml tree using `keep_xml_tree=True`).  We'll load these annotations and map them to `Ngram` objects over our parsed sentences, that way we can easily compare our extracted candidate set with the gold annotations.  The code is fairly simple (see `tutorial/util.py`); note that we filter to only keep _disease_ annotations, and that the candidates should be uniquely identified by their `id` attribute:

In [26]:
from utils import collect_pubtator_annotations
gold = []
for doc, sents in corpus:
    gold += [a for a in collect_pubtator_annotations(doc, sents) if a.metadata['type'] == 'Disease']
gold = frozenset(gold)

Now, we have a set of gold annotations of the same type as our candidates (`Ngram`), and can use set operations (where candidate objects are hashed by their `id` attribute), e.g.:

In [27]:
len(gold.intersection(c.get_candidates()))

3039

For convenience, we'll use a basic helper method of the `Candidates` object:

In [28]:
c.gold_stats(gold)

# of gold annotations	= 4244
# of candidates		= 6944
Candidate recall	= 0.716
Candidate precision	= 0.438


We note that our focus in this stage is on **acheiving high candidate recall, without considering an impractically large candidate set**.  Our main focus after this stage will be on training a classifier to select which candidates are true; this will raise precision while hopefully keeping recall high.  _Note however that candidate recall is an upper bound for the recall of this classifier!_

So, we have some work to do.

## Using the `Viewer` to inspect data

Next, we'll use the `Viewer` class--here, specifically, the `SentenceNgramViewer`--to inspect the data.

To start, we'll assemble a random set of all the sentences where there are gold annotations _not in our candidate set_, i.e. where we missed something, and then inspect these in the `Viewer`:

In [29]:
from collections import defaultdict
from random import shuffle

# Index the gold annotations by sentence id
gold_by_sid = defaultdict(list)
for g in gold:
    gold_by_sid[g.sent_id].append(g)

# Get sentences
view_sents = [s for s in corpus.get_sentences() if len(c.get_candidates_in(s.id)) < len(gold_by_sid[s.id])]
shuffle(view_sents)
view_sents = view_sents[:50]

Now, we instantiate and render the `Viewer` object; note we're being a bit sloppy, passing in _all_ the candidates and gold labels, but the `Viewer` object will take care of indexing them by sentence, and will only render the sentences we pass in:

In [30]:
from ddlite_viewer import SentenceNgramViewer
sv = SentenceNgramViewer(view_sents, c.get_candidates(), gold=gold)
sv.render(n_per_page=3, height=225)

Viewer 
 
 
 
 
 
 The tic-like symptoms resolved completely after we reduced the dose of amisulpride down to 800 mg per day. 
 
 Moreover, PREGS is able to reverse the amnesic -like effects of NMDAR and GABA(A)R ligands. 
 
 However, the replacement of last two doses of mesna with saline or all of the mesna doses with dexamethasone did not prevent HC . 
 
 
 
 
 
 In our patient, there was no other etiology identified that could explain QT prolongation or TDP The complete disappearance of NSVT and premature ventricular contractions followed by normalization of QT interval after the drug was stopped strongly suggests fluconazole as the etiology. 
 
 Placebo-level incidence of extrapyramidal symptoms ( EPS ) with quetiapine in controlled studies of patients with bipolar mania . 
 
 Thromboembolic and other complications of oral contraceptive therapy in relationship to pretreatment levels of blood coagulation factors: summary report of a ten-year study. 
 
 
 
 
 
 This report describes a case of cognitive deterioration resulting from prolonged use of DM. 
 
 When QTc interval prolongation is observed, early magnesium supplementation is advocated. 
 
 Decision-analytic techniques were used to estimate the cost of treating RIHA . 
 
 
 
 
 
 rTMS of supplementary motor area modulates therapy-induced dyskinesias in Parkinson disease . 
 
 All 3 patients had loss of central visual acuity, colour vision (Ishihara) and visual field . 
 
 We therefore conducted the present acute dose-response study in four 1-methyl-4-phenyl-1,2,3,6-tetrahydropyridine (MPTP)-exposed cynomolgus monkeys primed to exhibit levodopa-induced dyskinesias to evaluate the locomotor and dyskinetic effects on challenge with four doses (from 0.03 to 1.0 mg/kg) of A-86929 ([-]-[5aR,11bS]-4,5,5a,6,7,11b-hexahydro-2-propyl-3-thia-5-+ ++azacyclopent-1- ena[c]phenathrene-9-10-diol), a selective and full DA D1-like receptor agonist with an intermediate duration of action. 
 
 
 
 
 
 Future studies are needed to assess the therapeutic potential emerging from our finding for human W- ICH . 
 
 It is likely that the early and striking increase of plasma HDL found in nephrotic rats is related to a direct effect of the drug on HDL metabolism. 
 
 The incidence of discontinuations for hypertension -related and oedema -related AEs were significantly higher with etoricoxib (2.5% and 1.1% respectively) compared with diclofenac (1.5% and 0.4% respectively; p<0.001 for hypertension and p<0.01 for oedema ). 
 
 
 
 
 
 These results show that reserpine produces different and abnormal movements , which are related to dose and schedule employed and can be considered as parkinsonian -like and tardive dsykinesia signs. 
 
 CONCLUSIONS: High concordance in major developmental anomalies between Wistar and SD rats were noted with the exception of VSD in the SD rats and hydrocephalus in the Wistar rats. 
 
 OBJECTIVE: To quantify specific clinical predictors of reduction in renal function in patients with CHF who are prescribed angiotensin-converting enzyme inhibitor therapy. 
 
 
 
 
 
 Serum C3 complement levels were significantly reduced in ALF patients compared with control subjects (P < 0.0005). 
 
 Oral administration of KF17837 (2.5, 10.0 and 30.0 mg/kg) significantly ameliorated the cataleptic responses induced by intracerebroventricular administration of an adenosine A2A receptor agonist, CGS 21680 (10 micrograms), in a dose-dependent manner. 
 
 Symptom scores relating to the severity of constipation were lower in cisapride-treated constipation -predominant IBS patients [score, 54 +/- 5 versus 67 +/- 14 mm, cisapride versus placebo (P < 0.05); pretreatment, 62 +/- 19 mm]. 
 
 
 
 
 
 The dysrhythmias resolved after discontinuation of the drug. 
 
 Amisulpride related tic-like symptoms in an adolescent schizophrenic . 
 
 Among the subjects treated with methadone, 28% men and 32% women had prolonged QTc interval . 
 
 
 
 
 
 Within the same timeframe, no L-W rat developed a simi

## Composing a better candidate extractor

Let's try to increase our candidate recall using more of the `Matcher` operators and their functionalities.  First, let's turn on **Porter stemming** in our dictionary matcher; Porter stemming is an aggressive rules-based method for normalizing word endings.

In [31]:
# Define a new matcher
matcher = DictionaryMatch(d=diseases, longest_match_only=False, stemmer='porter')

# Extract a new set of candidates
%time c = Candidates(ngrams, matcher, corpus.get_sentences())
c.gold_stats(gold)

Extracting candidates...
CPU times: user 12.7 s, sys: 285 ms, total: 13 s
Wall time: 12.8 s
# of gold annotations	= 4244
# of candidates		= 9266
Candidate recall	= 0.749
Candidate precision	= 0.343


/Users/bradenhancock/anaconda/lib/python2.7/site-packages/nltk/stem/porter.py:274: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  if word[-1] == 's':


Next, note that *`Matcher` objects are compositional*. Observing in the `Viewer` that we are missing all of the acronyms, let's start with the `Union` operator, to integrate a dictionary for this:

In [32]:
from ddlite_matchers import Union
from load_dictionaries import load_acronym_dictionary

# Load the disease phrase dictionary
acronyms = load_acronym_dictionary()
print "Loaded %s acronyms!" % len(acronyms)

# Define a new matcher
matcher = Union(
    DictionaryMatch(d=diseases, longest_match_only=False, stemmer='porter'),
    DictionaryMatch(d=acronyms, ignore_case=False))

# Extract a new set of candidates
%time c = Candidates(ngrams, matcher, corpus.get_sentences())
c.gold_stats(gold)

Loaded 36904 acronyms!
Extracting candidates...
CPU times: user 14.2 s, sys: 107 ms, total: 14.3 s
Wall time: 14.3 s
# of gold annotations	= 4244
# of candidates		= 9811
Candidate recall	= 0.778
Candidate precision	= 0.336


Next, we try using the `Concat` and `RegexMatch` operators to find candidate mentions composed of an _adjective followed by a term matching our diseases dictionary_.  Note in particular that we set `left_required=False` so that exact matches to our dictionary (with no adjective prepended) will still work:

In [33]:
from ddlite_matchers import Concat, RegexMatchEach
matcher = Union(
    Concat(
        RegexMatchEach(rgx=r'JJ*', attrib='poses'),
        DictionaryMatch(d=diseases, longest_match_only=False, stemmer='porter'),
        left_required=False),
    DictionaryMatch(d=acronyms, ignore_case=False))

# Extract a new set of candidates
%time c = Candidates(ngrams, matcher, corpus.get_sentences())
c.gold_stats(gold)

Extracting candidates...
CPU times: user 27.2 s, sys: 416 ms, total: 27.6 s
Wall time: 27.4 s
# of gold annotations	= 4244
# of candidates		= 12292
Candidate recall	= 0.821
Candidate precision	= 0.284


### Running candidate extraction in parallel

Note that **the candidate extraction procedure can be parallelized across multiple cores using the `parallelism=N`** optional argument to the `Candidates` object.

### More coming here...

We've increased the candidate recall (on the development set) by 7.3% using some simple compositional `Matcher` operators.  We'll be adding more here soon!

## Connecting with the rest of the `DDLite` workflow

We are in the process of a big code refactor!  For now, to connect the candidates derived as in above to the rest of the `DDLite` workflow, create a `CandidateExtractor` class as follows:

In [ ]:
from ddlite_matchers import CandidateExtractor
ce = CandidateExtractor(ngrams, matcher)

This object can now be used in place of the candidate extractors in the other tutorials!